In [1]:
# default_exp utils

In [1]:
#hide
from nbdev.showdoc import *

In [1]:
# export
from loguru import logger
from functools import wraps
import mmcv
import inspect
ICACHE = dict()
#export
import xxhash
import pickle

def identify(x):
    '''Return an hex digest of the input'''
    return xxhash.xxh64(pickle.dumps(x), seed=0).hexdigest()


def memoize(func):
    import os
    import pickle
    from functools import wraps
    import xxhash
    '''Cache result of function call on disk
    Support multiple positional and keyword arguments'''
    @wraps(func)
    def memoized_func(*args, **kwargs):
        cache_dir = '.cache'
        try:
            import inspect
            func_id = identify((inspect.getsource(func), args, kwargs))
            cache_path = os.path.join(cache_dir, func.__name__+'_'+func_id)

            if (os.path.exists(cache_path) and
                    not func.__name__ in os.environ and
                    not 'BUST_CACHE' in os.environ):
                result = pickle.load(open(cache_path, 'rb'))
            else:
                result = func(*args, **kwargs)
                os.makedirs(cache_dir, exist_ok=True)
                pickle.dump(result, open(cache_path, 'wb'))
            return result
        except (KeyError, AttributeError, TypeError, Exception) as e:
            logger.warning(f'Exception: {e}, use default function call')
            return func(*args, **kwargs)
    return memoized_func


def imemoize(func):
    """
        Memoize a function into memory, the function recaculate only 
        change when its belonging arguments change
    """
    @wraps(func)
    def _f(*args, **kwargs):

        timer = mmcv.Timer()

        ident_name = identify((inspect.getsource(func), args, kwargs))
        # if not ident_name in ICACHE:
        try:
            result = ICACHE[ident_name]
        except:
            result = func(*args, **kwargs)
            # logger.info('Imemoize {}, firsttime:  runtime: {:0.2f} s'.format(
            #     func.__name__, timer.since_last_check()))
            ICACHE[ident_name] = result
        # else:
            
            # logger.info('Imemoize {} cached, runtime: {:0.2f} s'.format(
            #     func.__name__, timer.since_last_check()))

        return result
    return _f

In [3]:
import time
@imemoize
def example_function(x):
    x = x+1
    time.sleep(5)
    return x

x = example_function(0)
x = example_function(0)

2022-06-04 03:56:20.773 | INFO     | __main__:_f:58 - Imemoize example_function, firsttime:  runtime: 5.01 s
2022-06-04 03:56:20.775 | INFO     | __main__:_f:63 - Imemoize example_function cached, runtime: 0.00 s


In [4]:
example_function  = memoize(example_function)

# BUILD

In [3]:
!nbdev_build_lib

Converted 00_visualize.ipynb.
Converted 01_process.ipynb.
Converted 03_1_memoize.ipynb.
Converted 03_utils.ipynb.
Converted 04_debug.ipynb.
Converted 05_coco_dataset.ipynb.
Converted 06_cli.ipynb.
Converted all.ipynb.
Converted index.ipynb.
